<a href="https://colab.research.google.com/github/mjplacroix/lambda_project_1/blob/master/Sprint_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data importing and Cleaning

- import
- removed commas for price conversion from string object to float
- converted Date to datetime
- converted datetime to separate colums
- converted Transation Fee to float
- split into train - val - test along datetime
- dropped datetime column
- set target and X features matrix and y target vector

In [1]:
import pandas as pd
import numpy as np

eth_df = pd.read_csv('https://raw.githubusercontent.com/mjplacroix/lambda_project_1/master/ETH_master_dataset%20(1).csv')
print('Original shape:', eth_df.shape)

Original shape: (1645, 19)


In [0]:
# # index 891 - 918 - problem of commas
# print(eth_df['Price'][890:950])

# removing commas by replacing them with nothing 
eth_df['Price'] = eth_df['Price'].str.replace(',', '')
# converting 'Price' from an object to a float
eth_df['Price'] = eth_df['Price'].astype(float)

In [0]:
# converting Date to a pandas datetime
eth_df['Date(UTC)'] = pd.to_datetime(eth_df['Date(UTC)'], infer_datetime_format=True)

In [0]:
# converting datetime to indvidual columsn of year - month - day for analysis
# and pipelines 
eth_df['Year'] = pd.to_datetime(eth_df['Date(UTC)']).dt.year
eth_df['Month'] = pd.to_datetime(eth_df['Date(UTC)']).dt.month
eth_df['Day'] = pd.to_datetime(eth_df['Date(UTC)']).dt.day

In [5]:
eth_df.dtypes

Date(UTC)             datetime64[ns]
NetworkUtilization           float64
TransactionFee                object
BlockDifficulty              float64
NetworkHash                  float64
Uncles                         int64
BlockCountRewards              int64
BlockReward                  float64
GasUsed                        int64
GasLimit                       int64
Value (Wei)                    int64
BlockTime                    float64
BlockSize                      int64
AddressCount                   int64
TxGrowth                       int64
Ethersupply2                 float64
Supply                       float64
MarketCap                    float64
Price                        float64
Year                           int64
Month                          int64
Day                            int64
dtype: object

In [0]:
# converting 'TransactionFee' from an object to a float
eth_df['TransactionFee'] = eth_df['TransactionFee'].astype(float)

## Feature Engineering

- column with tomorrow's price as that's the real predictive value
 - DONE

In [0]:
length = range(1, len(eth_df['Price']))
tom_price = []

for i in length:
  Price = eth_df.iloc[i]['Price']
  tom_price.append(Price)

tom_price.append(float(184.47))

In [8]:
len(eth_df['Price']), len(tom_price)

(1645, 1645)

In [0]:
eth_df['Price_tomorrow'] = tom_price

- numeric increase/decrease from day before

In [0]:
length = range(1, len(eth_df['Price']))
#length = range(1, 20)
price_change_amount = [0]

for i in length:
  Price = eth_df.iloc[i]['Price']
  Yest_price = eth_df.iloc[i-1]['Price']
  change = Price - Yest_price
  price_change_amount.append(change)

In [0]:
eth_df['price_change_amount'] = price_change_amount

In [12]:
eth_df.head(20)

,Date(UTC),NetworkUtilization,TransactionFee,BlockDifficulty,NetworkHash,Uncles,BlockCountRewards,BlockReward,GasUsed,GasLimit,Value (Wei),BlockTime,BlockSize,AddressCount,TxGrowth,Ethersupply2,Supply,MarketCap,Price,Year,Month,Day,Price_tomorrow,price_change_amount
0,2015-07-30,0.0000,0.000000e+00,0.121,11.5297,1299,6912,39316.09375,0,0,0,4.46,644,9205,8893,7.204931e+07,7.204931e+07,0.000000,0.00,2015,7,30,0.00,0.00
1,2015-07-31,0.0000,0.000000e+00,0.603,51.4594,511,6863,36191.71875,0,0,0,12.58,582,9361,0,7.208550e+07,7.208550e+07,0.000000,0.00,2015,7,31,0.00,0.00
2,2015-08-01,0.0000,0.000000e+00,0.887,57.7845,334,5293,27705.93750,0,0,0,16.34,575,9476,0,7.211320e+07,7.211320e+07,0.000000,0.00,2015,8,1,0.00,0.00
3,2015-08-02,0.0000,0.000000e+00,1.020,67.9224,394,5358,28223.43750,0,0,0,16.12,581,9563,0,7.214143e+07,7.214143e+07,0.000000,0.00,2015,8,2,0.00,0.00
4,2015-08-03,0.0000,0.000000e+00,1.126,74.5737,443,5280,27976.71875,0,0,0,16.37,587,9639,0,7.216940e+07,7.216940e+07,0.000000,0.00,2015,8,3,0.00,0.00
5,2015-08-04,0.0000,0.000000e+00,1.217,82.0352,458,5368,28478.43750,0,0,0,16.10,587,9696,0,7.219788e+07,7.219788e+07,0.000000,0.00,2015,8,4,0.00,0.00
6,2015-08-05,0.0000,0.000000e+00,1.328,86.1558,371,5235,27528.59375,0,0,0,16.51,579,9749,0,7.222541e+07,7.222541e+07,0.000000,0.00,2015,8,5,0.00,0.00
7,2015-08-06,0.0000,0.000000e+00,1.381,88.3326,411,5116,27075.46875,0,0,0,16.89,584,9790,0,7.225249e+07,7.225249e+07,0.000000,0.00,2015,8,6,2.77,0.00
8,2015-08-07,0.1317,3.731841e+19,1.471,95.2981,409,5189,27437.65625,49353826,122980,604684154870,16.65,633,10314,2050,7.227992e+07,7.227992e+07,200.215391,2.77,2015,8,7,0.81,2.77
9,2015-08-08,0.0468,6.809997e+19,1.586,104.8971,458,5256,27943.43750,376006093,1940277,322713574989,16.44,668,10730,2881,7.230787e+07,7.230787e+07,58.569373,0.81,2015,8,8,0.74,-1.96


- simple binary increase/decrease 1-or-2

In [0]:
length = range(1, len(eth_df['Price']))
price_change = [0]

for i in length:
  price = eth_df.iloc[i]['price_change_amount']
  if price <= 0:
    price_change.append(0)
  else:
    price_change.append(1)

# 184.47 - 173.55
#price_change.append(1)

In [0]:
eth_df['price_change'] = price_change

- magnititude/percent of increase/decrease

In [15]:
length = range(1, len(eth_df['Price']))
Percent = [0]

for i in length:
  price_change = eth_df.iloc[i]['price_change_amount']
  price = eth_df.iloc[i]['Price']
  percent_change = (price_change / price) * 100
  Percent.append(round(percent_change, 2))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


In [16]:
len(eth_df['Price']), len(Percent)

(1645, 1645)

In [0]:
eth_df['Percent'] = Percent

In [18]:
eth_df.head(20)

,Date(UTC),NetworkUtilization,TransactionFee,BlockDifficulty,NetworkHash,Uncles,BlockCountRewards,BlockReward,GasUsed,GasLimit,Value (Wei),BlockTime,BlockSize,AddressCount,TxGrowth,Ethersupply2,Supply,MarketCap,Price,Year,Month,Day,Price_tomorrow,price_change_amount,price_change,Percent
0,2015-07-30,0.0000,0.000000e+00,0.121,11.5297,1299,6912,39316.09375,0,0,0,4.46,644,9205,8893,7.204931e+07,7.204931e+07,0.000000,0.00,2015,7,30,0.00,0.00,0,0.00
1,2015-07-31,0.0000,0.000000e+00,0.603,51.4594,511,6863,36191.71875,0,0,0,12.58,582,9361,0,7.208550e+07,7.208550e+07,0.000000,0.00,2015,7,31,0.00,0.00,0,NaN
2,2015-08-01,0.0000,0.000000e+00,0.887,57.7845,334,5293,27705.93750,0,0,0,16.34,575,9476,0,7.211320e+07,7.211320e+07,0.000000,0.00,2015,8,1,0.00,0.00,0,NaN
3,2015-08-02,0.0000,0.000000e+00,1.020,67.9224,394,5358,28223.43750,0,0,0,16.12,581,9563,0,7.214143e+07,7.214143e+07,0.000000,0.00,2015,8,2,0.00,0.00,0,NaN
4,2015-08-03,0.0000,0.000000e+00,1.126,74.5737,443,5280,27976.71875,0,0,0,16.37,587,9639,0,7.216940e+07,7.216940e+07,0.000000,0.00,2015,8,3,0.00,0.00,0,NaN
5,2015-08-04,0.0000,0.000000e+00,1.217,82.0352,458,5368,28478.43750,0,0,0,16.10,587,9696,0,7.219788e+07,7.219788e+07,0.000000,0.00,2015,8,4,0.00,0.00,0,NaN
6,2015-08-05,0.0000,0.000000e+00,1.328,86.1558,371,5235,27528.59375,0,0,0,16.51,579,9749,0,7.222541e+07,7.222541e+07,0.000000,0.00,2015,8,5,0.00,0.00,0,NaN
7,2015-08-06,0.0000,0.000000e+00,1.381,88.3326,411,5116,27075.46875,0,0,0,16.89,584,9790,0,7.225249e+07,7.225249e+07,0.000000,0.00,2015,8,6,2.77,0.00,0,NaN
8,2015-08-07,0.1317,3.731841e+19,1.471,95.2981,409,5189,27437.65625,49353826,122980,604684154870,16.65,633,10314,2050,7.227992e+07,7.227992e+07,200.215391,2.77,2015,8,7,0.81,2.77,1,100.00
9,2015-08-08,0.0468,6.809997e+19,1.586,104.8971,458,5256,27943.43750,376006093,1940277,322713574989,16.44,668,10730,2881,7.230787e+07,7.230787e+07,58.569373,0.81,2015,8,8,0.74,-1.96,0,-241.98


- streak?
- MA?
- consider removing 'Market Cap' as that too closely mimics the price

In [19]:
# splitting into train (1315), validate (165), and test (165)
train = eth_df[eth_df['Date(UTC)'] < '2019-03-01']
val  = eth_df[eth_df['Date(UTC)'] >= '2019-03-01']
test = val[val['Date(UTC)'] > '2019-08-15']
val = val[val['Date(UTC)'] <= '2019-08-15']
train.shape, val.shape, test.shape

((1310, 26), (168, 26), (167, 26))

In [0]:
# dropping datetime column
train = train.drop(['Date(UTC)'], axis=1)
val = val.drop(['Date(UTC)'], axis=1)
test = test.drop(['Date(UTC)'], axis=1)

In [0]:
# dropping MarketCap column since it too closely mimics price
train = train.drop(['MarketCap'], axis=1)
val = val.drop(['MarketCap'], axis=1)
test = test.drop(['MarketCap'], axis=1)

In [22]:
train.head()

,NetworkUtilization,TransactionFee,BlockDifficulty,NetworkHash,Uncles,BlockCountRewards,BlockReward,GasUsed,GasLimit,Value (Wei),BlockTime,BlockSize,AddressCount,TxGrowth,Ethersupply2,Supply,Price,Year,Month,Day,Price_tomorrow,price_change_amount,price_change,Percent
0,0.0,0.0,0.121,11.5297,1299,6912,39316.09375,0,0,0,4.46,644,9205,8893,7.204931e+07,7.204931e+07,0.0,2015,7,30,0.0,0.0,0,0.0
1,0.0,0.0,0.603,51.4594,511,6863,36191.71875,0,0,0,12.58,582,9361,0,7.208550e+07,7.208550e+07,0.0,2015,7,31,0.0,0.0,0,NaN
2,0.0,0.0,0.887,57.7845,334,5293,27705.93750,0,0,0,16.34,575,9476,0,7.211320e+07,7.211320e+07,0.0,2015,8,1,0.0,0.0,0,NaN
3,0.0,0.0,1.020,67.9224,394,5358,28223.43750,0,0,0,16.12,581,9563,0,7.214143e+07,7.214143e+07,0.0,2015,8,2,0.0,0.0,0,NaN
4,0.0,0.0,1.126,74.5737,443,5280,27976.71875,0,0,0,16.37,587,9639,0,7.216940e+07,7.216940e+07,0.0,2015,8,3,0.0,0.0,0,NaN


In [23]:
# The Price column is the target
target = 'Price_tomorrow'

# filtering out target
train_features = train.drop(columns=[target])
features = train_features.columns.tolist()

# separating in X features matrix and y target vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

X_train.shape, X_val.shape, X_test.shape

((1310, 23), (168, 23), (167, 23))

In [24]:
train.head()

,NetworkUtilization,TransactionFee,BlockDifficulty,NetworkHash,Uncles,BlockCountRewards,BlockReward,GasUsed,GasLimit,Value (Wei),BlockTime,BlockSize,AddressCount,TxGrowth,Ethersupply2,Supply,Price,Year,Month,Day,Price_tomorrow,price_change_amount,price_change,Percent
0,0.0,0.0,0.121,11.5297,1299,6912,39316.09375,0,0,0,4.46,644,9205,8893,7.204931e+07,7.204931e+07,0.0,2015,7,30,0.0,0.0,0,0.0
1,0.0,0.0,0.603,51.4594,511,6863,36191.71875,0,0,0,12.58,582,9361,0,7.208550e+07,7.208550e+07,0.0,2015,7,31,0.0,0.0,0,NaN
2,0.0,0.0,0.887,57.7845,334,5293,27705.93750,0,0,0,16.34,575,9476,0,7.211320e+07,7.211320e+07,0.0,2015,8,1,0.0,0.0,0,NaN
3,0.0,0.0,1.020,67.9224,394,5358,28223.43750,0,0,0,16.12,581,9563,0,7.214143e+07,7.214143e+07,0.0,2015,8,2,0.0,0.0,0,NaN
4,0.0,0.0,1.126,74.5737,443,5280,27976.71875,0,0,0,16.37,587,9639,0,7.216940e+07,7.216940e+07,0.0,2015,8,3,0.0,0.0,0,NaN


## Initial Baselines

- Simple majority mean

In [25]:
from sklearn.metrics import mean_absolute_error

y_pred = []
y_pred.append(y_train.mean())
y_pred = y_pred * len(train)
MAE = mean_absolute_error(y_train, y_pred)
print(f"Majority Train MAE: {MAE:.2f}")

y_pred = []
y_pred.append(y_train.mean())
y_pred = y_pred * len(y_val)
mae = mean_absolute_error(y_val, y_pred)
print(f'Majority Val MAE: {mae:.2f}')

Majority Train MAE: 206.93
Majority Val MAE: 48.64


In [0]:
# unsure why this is causing an error - can it only do binary/classification?

# # Accuracy Score 
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_val, y_pred)

- Linear Regression
 - Metrics: MSA, RMSA, R^2

- Trees?
- Forest?
- XGDBoost?

Etherscan API - https://etherscan.io/apis

Messario API - https://messari.io/api

### Interesting exploratory question. Will I get a different model or modeling accuracy if I move the decimal right twice to convert everything to whole dollars (integers)?